In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
nrows=50000

In [ ]:
train_data=pd.read_csv("../input/amex-default-prediction/train_data.csv",nrows=nrows)

In [ ]:
train_labels=pd.read_csv("../input/amex-default-prediction/train_labels.csv",nrows=nrows)

In [ ]:
test_data=pd.read_csv("../input/amex-default-prediction/test_data.csv",nrows=nrows)

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
train_labels.head()

In [ ]:
train=pd.merge(train_data,train_labels,how="inner")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
train["target"].unique

In [ ]:
np.unique(train["target"])

In [ ]:
len(train["customer_ID"])

In [ ]:
len(train["target"])

In [ ]:
train.head()

In [ ]:
cols = train.columns
num_cols=train._get_numeric_data().columns


In [ ]:
list(set(cols) - set(num_cols))

In [ ]:
train[["S_2","customer_ID","D_64","D_63"]]

In [ ]:
pd.to_datetime(train['S_2']).astype('int64')

In [ ]:
train.S_2.max()

In [ ]:
type(pd.to_datetime(train['S_2']).astype('int64'))

In [ ]:

train['S_2'] = pd.to_datetime(train['S_2']).astype('int64')
max_a = train.S_2.max()
min_a = train.S_2.min()
min_norm = -1
max_norm =1


In [ ]:
train['S_2'] = (train.S_2- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm

In [ ]:
df_one_hot = pd.get_dummies(train, columns = ["D_64"], prefix = ["D_64"])

In [ ]:
df_one_hot.head()

In [ ]:
df_one_hot2 = pd.get_dummies(df_one_hot, columns = ["D_63"], prefix = ["D_63"])

In [ ]:
df_one_hot2.head()

In [ ]:
train=df_one_hot2

In [ ]:
train.dtypes

In [ ]:
df_one_hot3 = pd.get_dummies(train, columns = ["customer_ID"], prefix = ["customer_ID"])

In [ ]:
df_one_hot3.head()

In [ ]:
train=df_one_hot3

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(train)
X_train_scaled = scaler.transform(train)

In [ ]:
train.head()

In [ ]:
df = train.copy()
y = df["target"]
X = df.drop(['target'], axis=1)


In [ ]:
len(y)

In [ ]:
X.columns

In [ ]:
y.head()

In [ ]:
len(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier().fit(X_train, y_train)

In [ ]:
xgb_model

In [ ]:
len(X_train.columns)

In [ ]:
X_train.iloc[:,:4283]

In [ ]:
y_pred = xgb_model.predict(X_train)
accuracy_score(y_train, y_pred)

In [ ]:
y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
test_data.head()

In [ ]:
len(test_data)

In [ ]:
cols = test_data.columns
num_cols=train._get_numeric_data().columns

In [ ]:
list(set(cols) - set(num_cols))

In [ ]:
test_data[['customer_ID', 'D_64', 'D_63']]

In [ ]:
df_one_hot1 = pd.get_dummies(test_data, columns = ["D_63"], prefix = ["D_63"])

In [ ]:
df_one_hot2 = pd.get_dummies(df_one_hot1, columns = ["D_64"], prefix = ["D_64"])

In [ ]:
df_one_hot2.head()

In [ ]:
test = pd.get_dummies(df_one_hot2, columns = ["customer_ID"], prefix = ["customer_ID"])

In [ ]:
for i in test.columns:
    if i.startswith("D_63"):
        print("Yes")
    elif i.startswith("D_64"):
        print("Yes")
    else:
        print("No")

In [ ]:
""" OR """

In [ ]:
 set(["Yes" if i.startswith("D_63") else 'No' if i.startswith("D_64") else 'No' for i in test.columns])

In [ ]:
test.head()

In [ ]:
test['S_2'] = pd.to_datetime(test['S_2']).astype('int64')
max_a = test.S_2.max()
min_a = test.S_2.min()
min_norm = -1
max_norm =1


In [ ]:
test['S_2'] = (test.S_2- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm

In [ ]:
scaler.fit(test)
X_test_scaled = scaler.transform(test)

In [ ]:
xgb_model = XGBClassifier().fit(X_train.iloc[:,:4283], y_train)

In [ ]:
test_pred = xgb_model.predict(X_test_scaled)

In [ ]:
test.head()

In [ ]:
test_data["customer_ID"][0]

In [ ]:
test["customer_ID_00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7"]

In [ ]:
len(test_pred)

In [ ]:
test_data["customer_ID"]

In [ ]:
sub = ({
    'customer_ID':test_data["customer_ID"],
    'prediction' :test_pred
               })

In [ ]:
sub = pd.DataFrame(sub)

In [ ]:
sub.head()


In [ ]:
len(sub)

In [ ]:
sub=sub.groupby(["customer_ID"])["prediction"].agg('mean')

In [ ]:
sub.head()

In [ ]:
type(sub)

In [ ]:
sub.values

In [ ]:
sub.index

In [ ]:
sub=pd.DataFrame({'customer_ID':sub.index, 'prediction':sub.values})

In [ ]:
sub["prediction"]=sub.prediction.round()

In [ ]:
sub.head()

In [ ]:
len(sub)

In [ ]:
sub.to_csv('submission.csv', index=False)